<a href="https://colab.research.google.com/github/alma-scheffer/MovieRecommender/blob/main/MovieRecommender_AS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FINE TUNE / TODO**
- add filter for "genre" for target set! **future step**
- record a video, when it is working! / save it as back-up
- list abilities, currently possible;
- list abilities, currently NOT possible;
- dropped all lines with numVotes < 500 -> **improves speed**

# Import all necessary sources

In [ ]:
!pip install --quiet gradio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

import gradio as gr
import requests

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://docs.google.com/document/d/1VF8KSppXcbly7NbVds5LQqwLC8T0uGpDKZFseJGv3xE/edit

title_movies_fin is our clean and "light" dataset.

In [ ]:
# Okay, I cannot input from shared drive, so I use the one, which is in my drive. 
# If you copy the csv (saved also in the project file) and save in your drives most probably you can access this and play around. 
title_movies_fin = pd.read_csv('/content/drive/MyDrive/title_movies.csv')

In [ ]:
title_movies_fin = title_movies_fin.sort_values(by = "numVotes", ascending = False)

In [ ]:
title_movies_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246361 entries, 62788 to 169714
Data columns (total 41 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      246361 non-null  int64  
 1   title_id        246361 non-null  object 
 2   titleType       246361 non-null  object 
 3   primaryTitle    246361 non-null  object 
 4   originalTitle   246361 non-null  object 
 5   isAdult         246361 non-null  int64  
 6   startYear       246361 non-null  float64
 7   runtimeMinutes  246361 non-null  float64
 8   genres          246361 non-null  object 
 9   averageRating   246361 non-null  float64
 10  numVotes        246361 non-null  float64
 11  Action          246361 non-null  int64  
 12  Adult           246361 non-null  int64  
 13  Adventure       246361 non-null  int64  
 14  Animation       246361 non-null  int64  
 15  Biography       246361 non-null  int64  
 16  Comedy          246361 non-null  int64  
 17  Crime 

In [ ]:
title_movies_fin.head()

In [ ]:
title_movies_fin.columns

Index(['Unnamed: 0', 'title_id', 'titleType', 'primaryTitle', 'originalTitle',
       'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'averageRating',
       'numVotes', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'other', 'actors'],
      dtype='object')

In [ ]:
## MAYBE TO DROP ROWS WITH NUMVOTES < 500 OR < 1000
# 500: 53 k rows remain
# 1000: 36 k rows remain
# title_movies_fin[title_movies_fin["numVotes"] >500].shape

In [ ]:
title_movies_fin = title_movies_fin[title_movies_fin["numVotes"] >500]
# This is already significantly improving recommendation speed: from ~ 30 sec. reduces to 1 -2 sec.

# Create the KNN modell

First create the KNN modell, while using n_neighbors = 5 - > we can create 4 movies to watch. (1st item is itself.)

In [ ]:
# First create the modell on the numerical columns:
X = title_movies_fin[['isAdult', 'startYear', 'runtimeMinutes',  'averageRating', 'numVotes', 'Action', 'Adult', \
                      'Adventure', 'Animation', 'Biography','Comedy', 'Crime', 'Documentary', 'Drama', 'Family', \
                      'Fantasy','Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', \
                      'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', \
                      'War', 'Western', 'other']]
# Target is to provide a primaryTitle / or a list of titles
y = title_movies_fin["title_id"]

# Use scaler to standardize
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# Fit / train the modell on X_scaled
distanceKNN = NearestNeighbors(n_neighbors=5).fit(X_scaled)

# Create the recommendation function and the interface for the user to interact

In [ ]:
"""
# I needed to adjust the index to access the movietitle, because seems that when I saved the csv
# I have created a new column with indexes "Unnamed"

def replace(movieString):
  # Check if we are searching on id:
  if movieString[0:2] == "tt":

    # First get the needed columns in X_scaled for the movie, for which we want to find the neighbours
    neighborsearchID = X_scaled[title_movies_fin["title_id"] == movieString]
    messageID = "You should check out the following movies: " +  "\n"
    
    # Locate the neighbors place - > access their name and add to the message for the user
    for i in range(1,5):
      messageID += title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][3] + "\n"
    return movieString.replace(movieString, messageID)
  
  # Or we are searching on primaryTitle:
  else:
    
    neighborsearchTitle = X_scaled[title_movies_fin["primaryTitle"].str.contains(movieString, case = True)]
    messageTitle = "You should check out the following movies: " +  "\n"
    
    for i in range(1,5):
      messageTitle += title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle][0])[1][0][i]][3] + "\n"
    return movieString.replace(movieString, messageTitle)

# The interface: adding the title and the introduction paragraph
iface = gr.Interface(fn = replace, inputs=["textbox"], outputs="textbox",
    title="MOVIE RECOMMENDER",
    description="Type the title or the ID of your favorite movie from IMDB \
    and we recommend you 4 other movies, which you might like." + "\n" + \
    "Enjoy playing around and have a great MovieMarathon!", 
    theme = "darkhuggingface",
)

iface.launch()"""


# Second version, with the feature to specify year

This version contains the following enhanced features:
- Added year as filter to specify if multiple searches.
- Added error messages, where the search fails.
- Another message could be added, if there are multiple results to specify / instead of automatically choosing the first. 
- Provides result also for not 100% match.

In [ ]:
# Not needed I just wanted to check the steps for the correct way of filtering / what kind of input I need for the X_scaled
# (title_movies_fin["primaryTitle"] == "Bohemios") & (title_movies_fin["startYear"] == 1905)

In [ ]:
# Same, not needed, but to check, how the non-existing items search result looks 
# neighborsearchID = X_scaled[title_movies_fin["primaryTitle"] == "Bohemian"]

In [ ]:
# According to documentation you can create a dropdown menu, which sounds great.

In [ ]:
#@title
# I needed to adjust the index to access the movietitle, because seems that when I saved the csv
# I have created a new column with indexes "Unnamed"
"""
def replace(movieString, movieYear, chooseGenre):
  # Check if movieYear filled:
  if movieYear == "":

    # Check if we are searching on id:
    if movieString[0:2] == "tt":

      # First get the needed columns in X_scaled for the movie, for which we want to find the neighbours
      neighborsearchID = X_scaled[title_movies_fin["title_id"] == movieString]

      # If the search resulted on empty df, provide the feedback for the user.
      if neighborsearchID.size == 0:
        messageError1 = "Your movie is not in our database: please re-check your input."
        return movieString.replace(movieString, messageError1)

      # Else print the nearest neighbors
      else: 
        messageID = "You should check out the following movies: " +  "\n"
    
      # Locate the neighbors place - > access their name and add to the message for the user
        for i in range(1,5):
          messageID += title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][3] + "\n"
        return movieString.replace(movieString, messageID)

    # Or we are searching on primaryTitle:
    else:
      neighborsearchTitle1 = X_scaled[title_movies_fin["primaryTitle"].str.contains(movieString, case = True)]
      
      if neighborsearchTitle1.size == 0:
        messageError2 = "Your movie is not in our database: please re-check your input."
        return movieString.replace(movieString, messageError2)
      
      else:
        messageTitle1 = "You should check out the following movies: " +  "\n"
    
        for i in range(1,5):
          messageTitle1 += title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle1][0])[1][0][i]][3] + "\n"
        return movieString.replace(movieString, messageTitle1)


  # If the user added correctly numeric data for movieYear:
  elif movieYear.isnumeric() == True:
    movieYear = int(movieYear)

    # Check if movieString, either id or title is filled:
    if movieString == "":

      messageTitle2 = "You have not chosen a movie: please help us to recommmend."
      return movieString.replace(movieString, messageTitle2)


    # We can improve our search in the database:
    else:
      neighborsearchTitle2 = X_scaled[(title_movies_fin["primaryTitle"].str.contains(movieString, case = True)) & (title_movies_fin["startYear"] == movieYear)]
      if neighborsearchTitle2.size == 0:
        messageError3 = "Your movie is not in our database: please re-check your input."
        return movieString.replace(movieString, messageError3)
      else:
        messageTitle2 = "You should check out the following movies: " +  "\n"
    
        for i in range(1,5):
          messageTitle2 += title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle2][0])[1][0][i]][3] + "\n"
        return movieString.replace(movieString, messageTitle2)
  
  # If movieYear is neither numeric, nor empty, then the input is not correct, so send an error message:
  else:
    messageError = ("You are trying to input invalid information")
    return movieString.replace(movieString, messageError)

    
# The interface: adding the title and the introduction paragraph
iface = gr.Interface(fn = replace, inputs=["textbox", "textbox"], outputs="textbox",
    title="MOVIE RECOMMENDER",
    description="Type the title or the ID of your favorite movie from IMDB \
    and we recommend you 4 other movies, which you might like." + "\n" + \
    "Enjoy playing around and have a great MovieMarathon!", 
    theme = "darkhuggingface"
)

iface.launch()
"""

# Actual version: movie (id, title, originalTitle), actor, genre, year filters

https://github.com/gradio-app/gradio/blob/master/demo/titanic_survival.py

In [ ]:
# So basically you can also filter the database with "genre"
# with the following code // it is better to use directly the genres column than the dummies
# title_movies_fin[title_movies_fin["genres"].str.contains("Documentary")]

In [ ]:
# We can also generate the genreTypes from title_movies_fin.columns[11:], while filtering only colum
genreTypes = ['War','Adult','Sci-Fi','Talk-Show','Animation','Musical','Family','Sport','Adventure','News','Game-Show','Short','Comedy','Action','History','Biography','Drama','Crime','Film-Noir','Romance','Documentary','Thriller','Western','','Mystery','Reality-TV','Fantasy','Horror','Music']
# sorted(genreTypes)

In [ ]:
# Let's do a different logic for the interface, where we split functions: 
# Filtering, to search for a movie and
# Either error message, if input is faulty
# Or recommendation

def replace(movieString, actor, genre, movieYear):
  # first set up the filters based on user input
  filterID = title_movies_fin["title_id"] == movieString
  filterTitle = (title_movies_fin["primaryTitle"].str.contains(movieString, case = False)) | \
  (title_movies_fin["originalTitle"].str.contains(movieString, case = False)) 
  
  # if I have not specified what to do with missing movieYear, the program was not running
  if movieYear == "":
    filterYear = True
  elif movieYear.isnumeric() == True:
    filterYear = title_movies_fin["startYear"] == int(movieYear)
  else:
    return "Year is not numeric, please re-check"
  filterGenre = title_movies_fin["genres"].str.contains(genre, case = True)
  filterActor = title_movies_fin["actors"].str.contains(actor, case = False)


  # After defining the filters we can check, if we search based on Id or title (primary or original)
  # Searching on ID means, first 2 character is tt
  if movieString[0:2] == "tt":
    
    # Saving the movie "features" from the standardized data set, for which we are searching for the neighbors
    neighborsearchID = X_scaled[filterID & filterYear & filterGenre & filterActor]

    # If the movie "feature" has an a shape(0,34), means that there was no result according to our criteria
    if neighborsearchID.shape == (0,34):
      return "I'm sorry, but there is no movie according your search criteria in our database"

    # If the result has a shape 1,34, means that there is exactly one row matching our search criteria  
    elif neighborsearchID.shape == (1,34):
      messageID1 = []
      
      # So we can recommend 4 other movies
      for i in range(1,5):
          messageID1.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][3])
          messageID1.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][40])
      return f"""You should check out the following movies: \n- {messageID1[0]} with {messageID1[1]}, \
      \n- {messageID1[2]} with {messageID1[3]}, \n- {messageID1[4]} with {messageID1[5]}, \
      \n- {messageID1[6]} with {messageID1[7]} """

    # Else we have multiple results and we wanna make it clear for the user, that we are not sure
    else:
      messageID2 = ["Did you like ", title_movies_fin[filterID & filterYear & filterGenre & filterActor].iloc[0][3], \
                    "with ", title_movies_fin[filterID & filterYear & filterGenre & filterActor].iloc[0][40]]
      for i in range(1,5):
          messageID2.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][3])
          messageID2.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchID][0])[1][0][i]][40])
      return f"""{messageID2[0]}{messageID2[1]}{messageID2[2]} {messageID2[3]}?\n\nThen you should check out the following \
movies: \n- {messageID2[4]} with {messageID2[5]}, \n- {messageID2[6]} with {messageID2[7]}, \n- {messageID2[8]} with {messageID2[9]},\
 \n- {messageID2[10]} with {messageID2[11]}"""

  # Here we search based on either primaryTitle or originalTitle
  else:
    # Same logic, define what you search in dataset and save the result in neighborsearch variable
    neighborsearchTitle = X_scaled[filterTitle & filterYear & filterGenre & filterActor]
    
    # Check if we have zero - one - or many matches and communicate accordingly
    if neighborsearchTitle.shape == (0,34):
      return "I'm sorry, but there is no movie according your search criteria in our database"
    elif neighborsearchTitle.shape == (1,34):
    #else:
      messageTitle1 = []
      for i in range(1,5):
          messageTitle1.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle][0])[1][0][i]][3])
          messageTitle1.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle][0])[1][0][i]][40])
      return f"""You should check out the following movies: \n- {messageTitle1[0]} with {messageTitle1[1]}, \
      \n- {messageTitle1[2]} with {messageTitle1[3]}, \n- {messageTitle1[4]} with {messageTitle1[5]}, \
      \n- {messageTitle1[6]} with {messageTitle1[7]}"""
    else:
      messageTitle2 = ["Did you like ", title_movies_fin[filterTitle & filterYear & filterGenre & filterActor].iloc[0][3], \
                    "with ", title_movies_fin[filterTitle & filterYear & filterGenre & filterActor].iloc[0][40]]
      for i in range(1,5):
          messageTitle2.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle][0])[1][0][i]][3])
          messageTitle2.append(title_movies_fin.iloc[distanceKNN.kneighbors([neighborsearchTitle][0])[1][0][i]][40])
      return f"""{messageTitle2[0]}{messageTitle2[1]} {messageTitle2[2]}{messageTitle2[3]}?\n\nThen you should check out \
the following movies: \n- {messageTitle2[4]} with {messageTitle2[5]}, \n- {messageTitle2[6]} with {messageTitle2[7]},\
\n- {messageTitle2[8]} with {messageTitle2[9]}, \n- {messageTitle2[10]} with {messageTitle2[11]}"""
     # return "I'm sorry, but your search is not specific enough. Maybe try again while providing Imdb id."
 
# The interface: adding the title and the introduction paragraph
iface = gr.Interface(replace,[gr.inputs.Textbox(), gr.inputs.Textbox(), gr.inputs.Dropdown(sorted(genreTypes)), gr.inputs.Textbox()],\
                     "text",
                     title="Movie recommender",\
                     description="Type the title or the ID of your favorite movie from IMDB \
                     and we recommend you 4 other movies, which you might like." + "\n" + \
                     "Enjoy playing around and have a great MovieMarathon!", \
                     theme = "darkhuggingface")

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://58034.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7874/',
 'https://58034.gradio.app')

In [ ]:
replace("", "", "Biography", "")

'Did you like The Wolf of Wall Street with Leonardo DiCaprio, Jonah Hill, Margot Robbie, Matthew McConaughey?\n\nThen you should check out the following movies: \n- Goodfellas with Robert De Niro, Ray Liotta, Joe Pesci, Lorraine Bracco, \n- The Godfather: Part II with Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton,\n- Catch Me If You Can with Leonardo DiCaprio, Tom Hanks, Christopher Walken, Martin Sheen, \n- The Departed with Leonardo DiCaprio, Matt Damon, Jack Nicholson, Mark Wahlberg'

In [ ]:
# For debugging: 
# print(replace("tt0111161", "1994", "Drama"))

In [ ]:
# For quick checking
# tt0111161
# tt0293927
# title_movies_fin[title_movies_fin["primaryTitle"].str.contains("Holiday")]

In [ ]:
# Obv.if for actors you check with "Tom Cruise"
# title_movies_fin[title_movies_fin["actors"].str.contains("Tom Cruise")]#.iloc[0][40]

In [ ]:
# For quick testing:
# title_movies_fin[title_movies_fin["title_id"] == "tt0111161"].iloc[0][40]

'Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler'

In [ ]:
# title_movies_fin[title_movies_fin["actors"].str.contains("Mel Gibson")]

In [ ]:
# Checking if combination for primaryTitle and originalTitle as filter criteria works
"""filterTitle = (title_movies_fin["primaryTitle"].str.contains("Bohemios", case = True)) | \
  (title_movies_fin["originalTitle"].str.contains("Bohemios", case = True)) 
filterTitle """

In [ ]:
# Checking the shape for movie features
# While playing with different criteria

# filterID = title_movies_fin["title_id"] == "tt0094868"
# filterTitle = title_movies_fin["primaryTitle"].str.contains(movieString, case = True)
# filterYear = True
# filterGenre = title_movies_fin["genres"].str.contains("Malibu", case = False)
# X_scaled[filterID & filterYear & filterGenre]

array([], shape=(0, 34), dtype=float64)

In [ ]:
# neighborsearch = X_scaled[filterID & filterYear & filterGenre]
# neighborsearch.shape

(0, 34)

In [ ]:
# Fine tuning the communication with the user
# title_movies_fin[filterTitle].iloc[0][3], title_movies_fin[filterTitle].iloc[0][6] 

('Bohemios', 1905)

In [ ]:
# title_movies_fin[title_movies_fin["primaryTitle"].str.contains("Star")]

Capabilities:
- you can enter a movie title, a movie id, a genre, year or actor names and you will get 4 movie recommended by our system;
- you can try even if you remember only partially at the movie title, what you liked!
- be careful, if you know the movie id the match has to be 100%! 

What this recommender cannot do currently:
- make recommendation based on favorite director, operator, producer;
- search on title, which are neither original, nor primary, i.e. Carmencita in Hungarian;
- search on language, region;
- read your mind - but sure, it tries;
- in case the input is very broad, we need more time to consider, what to recommend, it can go even up to 5 min, so be careful with your wishes;

Next steps: 
- include directors in search criteria,
- enable the user to select also for recommendation some features, like: hm, i like Tom Cruise and I wanna get recommended Dramas.